In [6]:
import pandas as pd 
import numpy as np 
import warnings 
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
ruta_archivo1 = r"C:\Users\roriv\DATA IMBALANCE\Customer-Churn.csv"

# Importar el archivo CSV en un DataFrame
df = pd.read_csv(ruta_archivo1)

In [4]:
df1= df.copy()
df1

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.6,Yes


In [24]:
df2= df.copy()

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  StreamingTV       7043 non-null   object 
 11  StreamingMovies   7043 non-null   object 
 12  Contract          7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7043 non-null   object 
 15  Churn             7043 non-null   object 
dtypes: float64(1), int64(2), object(13)
memory

In [7]:
df1['TotalCharges'] = pd.to_numeric(df1['TotalCharges'], errors='coerce')

In [10]:
print(df1.isnull().sum())
df1['TotalCharges'].fillna(df1['TotalCharges'].mean(), inplace=True)

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [11]:
selected_features = ["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]
X = df1[selected_features]

In [12]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, df1['Churn'], test_size=0.2, random_state=42)

In [14]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [15]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on the test data:", accuracy)

Accuracy on the test data: 0.8076650106458482


In [16]:
print(df1['Churn'].value_counts())

No     5174
Yes    1869
Name: Churn, dtype: int64


In [18]:
from sklearn.utils import resample
# Separate majority and minority classes
churn_majority = df1[df1['Churn'] == 'No']
churn_minority = df1[df1['Churn'] == 'Yes']

# Upsample minority class
churn_minority_upsampled = resample(churn_minority, 
                                   replace=True,     # Sample with replacement
                                   n_samples=len(churn_majority),  # Match the number of samples in the majority class
                                   random_state=42)  # Set a random seed for reproducibility

# Combine the upsampled minority class with the majority class
churn_upsampled = pd.concat([churn_majority, churn_minority_upsampled])

# Check the class distribution after upsampling
print(churn_upsampled['Churn'].value_counts())

No     5174
Yes    5174
Name: Churn, dtype: int64


In [20]:
# Separate majority and minority classes
churn_majority = df1[df1['Churn'] == 'No']
churn_minority = df1[df1['Churn'] == 'Yes']

# Downsample majority class
churn_majority_downsampled = resample(churn_majority, 
                                      replace=False,    # Sample without replacement
                                      n_samples=len(churn_minority),  # Match the number of samples in the minority class
                                      random_state=42)  # Set a random seed for reproducibility

# Combine the downsampled majority class with the minority class
churn_downsampled = pd.concat([churn_majority_downsampled, churn_minority])

# Check the class distribution after downsampling
print(churn_downsampled['Churn'].value_counts())

No     1869
Yes    1869
Name: Churn, dtype: int64


In [21]:
# Separate the features and target variable
X_upsampled = churn_upsampled[selected_features]
y_upsampled = churn_upsampled['Churn']

# Split the upsampled data into a training set and a test set
X_train_upsampled, X_test_upsampled, y_train_upsampled, y_test_upsampled = train_test_split(X_upsampled, y_upsampled, test_size=0.2, random_state=42)

# Initialize and train the logistic regression model on the upsampled training data
model_upsampled = LogisticRegression()
model_upsampled.fit(X_train_upsampled, y_train_upsampled)

# Predict on the upsampled test data
y_pred_upsampled = model_upsampled.predict(X_test_upsampled)

# Calculate accuracy on the upsampled test data
accuracy_upsampled = accuracy_score(y_test_upsampled, y_pred_upsampled)
print("Accuracy on the upsampled test data:", accuracy_upsampled)

Accuracy on the upsampled test data: 0.7314009661835749


In [22]:
# Separate the features and target variable
X_downsampled = churn_downsampled[selected_features]
y_downsampled = churn_downsampled['Churn']

# Split the downsampled data into a training set and a test set
X_train_downsampled, X_test_downsampled, y_train_downsampled, y_test_downsampled = train_test_split(X_downsampled, y_downsampled, test_size=0.2, random_state=42)

# Initialize and train the logistic regression model on the downsampled training data
model_downsampled = LogisticRegression()
model_downsampled.fit(X_train_downsampled, y_train_downsampled)

# Predict on the downsampled test data
y_pred_downsampled = model_downsampled.predict(X_test_downsampled)

# Calculate accuracy on the downsampled test data
accuracy_downsampled = accuracy_score(y_test_downsampled, y_pred_downsampled)
print("Accuracy on the downsampled test data:", accuracy_downsampled)

Accuracy on the downsampled test data: 0.7553475935828877


### Lab | Cross Validation

In [25]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier

# Apply SMOTE for upsampling
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_scaled, df2['Churn'])

# Split the SMOTE-upsampled data into a training set and a test set
X_train_smote, X_test_smote, y_train_smote, y_test_smote = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Initialize and train logistic regression model on SMOTE-upsampled training data
model_logistic_smote = LogisticRegression()
model_logistic_smote.fit(X_train_smote, y_train_smote)

# Predict on SMOTE-upsampled test data using logistic regression
y_pred_logistic_smote = model_logistic_smote.predict(X_test_smote)

# Calculate accuracy on SMOTE-upsampled test data using logistic regression
accuracy_logistic_smote = accuracy_score(y_test_smote, y_pred_logistic_smote)
print("Accuracy using Logistic Regression with SMOTE:", accuracy_logistic_smote)

# Initialize and train decision tree classifier model on SMOTE-upsampled training data
model_tree_smote = DecisionTreeClassifier(random_state=42)
model_tree_smote.fit(X_train_smote, y_train_smote)

# Predict on SMOTE-upsampled test data using decision tree classifier
y_pred_tree_smote = model_tree_smote.predict(X_test_smote)

# Calculate accuracy on SMOTE-upsampled test data using decision tree classifier
accuracy_tree_smote = accuracy_score(y_test_smote, y_pred_tree_smote)
print("Accuracy using Decision Tree Classifier with SMOTE:", accuracy_tree_smote)

Accuracy using Logistic Regression with SMOTE: 0.751207729468599
Accuracy using Decision Tree Classifier with SMOTE: 0.7632850241545893


In [26]:
from imblearn.under_sampling import TomekLinks

# Apply TomekLinks for downsampling
tomek = TomekLinks()
X_tomek, y_tomek = tomek.fit_resample(X_scaled, df2['Churn'])

# Split the TomekLinks-downsampled data into a training set and a test set
X_train_tomek, X_test_tomek, y_train_tomek, y_test_tomek = train_test_split(X_tomek, y_tomek, test_size=0.2, random_state=42)

# Initialize and train logistic regression model on TomekLinks-downsampled training data
model_logistic_tomek = LogisticRegression()
model_logistic_tomek.fit(X_train_tomek, y_train_tomek)

# Predict on TomekLinks-downsampled test data using logistic regression
y_pred_logistic_tomek = model_logistic_tomek.predict(X_test_tomek)

# Calculate accuracy on TomekLinks-downsampled test data using logistic regression
accuracy_logistic_tomek = accuracy_score(y_test_tomek, y_pred_logistic_tomek)
print("Accuracy using Logistic Regression with TomekLinks:", accuracy_logistic_tomek)

# Initialize and train decision tree classifier model on TomekLinks-downsampled training data
model_tree_tomek = DecisionTreeClassifier(random_state=42)
model_tree_tomek.fit(X_train_tomek, y_train_tomek)

# Predict on TomekLinks-downsampled test data using decision tree classifier
y_pred_tree_tomek = model_tree_tomek.predict(X_test_tomek)

# Calculate accuracy on TomekLinks-downsampled test data using decision tree classifier
accuracy_tree_tomek = accuracy_score(y_test_tomek, y_pred_tree_tomek)
print("Accuracy using Decision Tree Classifier with TomekLinks:", accuracy_tree_tomek)

Accuracy using Logistic Regression with TomekLinks: 0.7964804896710023
Accuracy using Decision Tree Classifier with TomekLinks: 0.7727620504973222
